In [10]:
!git clone https://github.com/yujiafeng8888/scCNA.git
%cd scCNA
!pip install .

致命错误：目标路径 'scCNA' 已经存在，并且不是一个空目录。
/Users/luoyilin/scCNA
Processing /Users/luoyilin/scCNA
  Preparing metadata (setup.py) ... done
  Created wheel for scCNA: filename=scCNA-0.1-py3-none-any.whl size=6090 sha256=d6ff727fd63e457952f6c4d4d2292dd59746a206f42a3373056d5f032e5437bd
  Stored in directory: /private/var/folders/rr/th9tmxcd7j37vgtqc9s41kch0000gn/T/pip-ephem-wheel-cache-bog5vh42/wheels/ed/74/bd/f65cf665b60ae248776493cf35d589094df200c6852942b544
Successfully built scCNA
  Attempting uninstall: scCNA
    Found existing installation: scCNA 0.1
    Uninstalling scCNA-0.1:
      Successfully uninstalled scCNA-0.1


In [1]:
!pip install scanpy

In [1]:
import numpy as np
import random
import scanpy as sc

In [2]:
import scanpy as sc
import scCNA as cna
import matplotlib.pyplot as plt
import infercnvpy as cnv
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score

In [3]:
adata = sc.read_h5ad("PBMC_simulated_cnas_041025.h5ad")

In [4]:

def simulate_cnas_windowed(
    adata,
    window_size=100,
    cna_types=("gain", "loss", "homo_del"),
    frequencies=(0.2, 0.5, 0.9),
    chroms=("1", "2", "3"),
    cell_key="cell_type"
):
    """
    Simulate CNAs in expression matrix by modifying expression of genomic windows.
    Writes `simulated_cnvs` to adata.obs (overwrites if exists).

    Parameters:
    - adata: AnnData object
    - window_size: number of genes per simulated CNA window
    - cna_types: list of CNA types to simulate
    - frequencies: list of cell-level frequencies for each simulated CNA
    - chroms: chromosomes to simulate CNAs on
    """
    if "simulated_cnvs" not in adata.obs.columns:
        adata.obs["simulated_cnvs"] = ""
    else:
        adata.obs["simulated_cnvs"] = ""

    if not isinstance(adata.X, np.ndarray):
        X = adata.X.toarray()
    else:
        X = adata.X

    var = adata.var
    simulated_log = []

    for chrom in chroms:
        genes_chr = var[var["chromosome"] == chrom]
        gene_indices = genes_chr.index.to_numpy()

        if len(gene_indices) < window_size:
            continue

        start_idx = random.randint(0, len(gene_indices) - window_size)
        window_genes = gene_indices[start_idx:start_idx + window_size]
        window_starts = var.loc[window_genes, "start"]
        region_start = window_starts.min()
        region_end = window_starts.max()

        # Choose CNA type and frequency
        cna_type = random.choice(cna_types)
        freq = random.choice(frequencies)
        n_cells = int(len(adata) * freq)
        selected_cells = np.random.choice(adata.obs_names, n_cells, replace=False)

        # Apply effect
        for cell in selected_cells:
            idx = adata.obs_names.get_loc(cell)
            if cna_type == "gain":
                X[idx, adata.var_names.isin(window_genes)] *= 1.5
            elif cna_type == "loss":
                X[idx, adata.var_names.isin(window_genes)] *= 0.5
            elif cna_type == "homo_del":
                X[idx, adata.var_names.isin(window_genes)] = 0

            region_str = f"{chrom}:{region_start}-{region_end} (CN {0 if cna_type == 'homo_del' else ('+' if cna_type == 'gain' else '-')})"
            if adata.obs.at[cell, "simulated_cnvs"] == "":
                adata.obs.at[cell, "simulated_cnvs"] = region_str
            else:
                adata.obs.at[cell, "simulated_cnvs"] += f";{region_str}"

        simulated_log.append((chrom, region_start, region_end, cna_type, freq))

    adata.X = X
    return adata, simulated_log


In [5]:
adata_sim, log = simulate_cnas_windowed(
    adata.copy(),
    window_size=100,
    cna_types=["gain", "loss", "homo_del"],
    frequencies=[0.2, 0.5, 0.9],
    chroms=["1", "2", "3"]
)


In [6]:
display(adata_sim.obs.head())

,n_genes_by_counts,total_counts,total_counts_ribo,pct_counts_ribo,total_counts_mt,pct_counts_mt,n_genes,n_counts,cell_type,simulated_cnvs
AAACCCAAGCGCCCAT-1,1005,1760.0,392.0,17.785845,52.0,2.359347,1005,1760.0,CD4 T cell,1:183472216.0-203626832.0 (CN 0)
AAACCCAAGGTTCCGC-1,4101,14240.0,4526.0,22.528622,1324.0,6.590343,4101,14240.0,Dendritic,1:183472216.0-203626832.0 (CN 0);2:85394753.0-...
AAACCCACAGAGTTGG-1,1742,4208.0,1043.0,17.726036,633.0,10.757988,1742,4208.0,CD14 monocyte,1:183472216.0-203626832.0 (CN 0);2:85394753.0-...
AAACCCACAGGTATGG-1,2122,4354.0,742.0,13.417721,434.0,7.848101,2122,4354.0,NK cell,2:85394753.0-98619106.0 (CN +)
AAACCCACATAGTCAC-1,1521,2819.0,1734.0,33.960049,553.0,10.830396,1521,2819.0,B cell,1:183472216.0-203626832.0 (CN 0);2:85394753.0-...


In [7]:
y_true = (adata_sim.obs['simulated_cnvs'].notna()) & (adata_sim.obs['simulated_cnvs'] != '')

In [8]:
print(y_true.value_counts())

simulated_cnvs
True     10038
False      271
Name: count, dtype: int64


In [9]:
ad_def0 = cna.find_cnas(
    adata_sim.copy(),
    reference_key='cell_type',
    reference_cat=[
        'CD4 T cell','CD14 monocyte','B cell','CD8 T cell',
        'NK cell','FCGR3A monocyte','Dendritic','Megakaryocyte'
    ],
    threshold=5,
    min_cells=20,
    window_size=100
)

/opt/anaconda3/lib/python3.12/site-packages/scanpy/preprocessing/_normalization.py:208: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


AxisError: axis 1 is out of bounds for array of dimension 1

In [13]:
ad_def = cna.find_cnas(
    adata.copy(),
    reference_key='cell_type',
    reference_cat=[
        'CD4 T cell','CD14 monocyte','B cell','CD8 T cell',
        'NK cell','FCGR3A monocyte','Dendritic','Megakaryocyte'
    ],
    threshold=5,
    min_cells=20,
    window_size=100
)

/opt/anaconda3/lib/python3.12/site-packages/scanpy/preprocessing/_normalization.py:208: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [14]:
y_true = (ad_sim.obs['simulated_cnvs'].notna()) & (ad_sim.obs['simulated_cnvs'] != '')
#y_pred = (ad_def.obs['detect_CNA'].notna()) & (ad_def.obs['detect_CNA'] != '')
y_pred = ad_def.obs['detect_CNA'].apply(lambda x: x != 'none' and isinstance(x, str))

print("=== Cell-level Binary Classification Metrics ===")
print("Accuracy :", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred))
print("Recall   :", recall_score(y_true, y_pred))
print("F1 Score :", f1_score(y_true, y_pred))
print("PR-AUC   :", average_precision_score(y_true, y_pred))

NameError: name 'ad_sim' is not defined